In [1]:
import transcriptic 
from transcriptic import commands, Container
from transcriptic.config import Connection
from transcriptic.jupyter import objects
import json
from pysd2cat.data import tx_fcs
from pysd2cat.data import pipeline
from pysd2cat.analysis import biofab_live_dead_analysis as blda
from pysd2cat.data import biofab_live_dead as bld


from os.path import expanduser
import requests
import os
import zipfile
import shutil
import fnmatch
import glob
import pandas as pd
import ast

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

Connection.from_file("~/.transcriptic")

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
#protocol = objects.protocol('pr1drp6hzrchh79')
[x['name'] for x in transcriptic.api.get_protocols() if x['id'] == 'pr1drp6hzrchh79']

[]

In [3]:
container_string = '423021.3' #'422936' #
results = transcriptic.api.inventory(container_string)


In [4]:
results

{'results': [{'id': 'ct1dqvp6nd7gej7',
   'container_type_id': '96-pcr',
   'barcode': '58304253',
   'deleted_at': None,
   'created_at': '2019-10-30T13:28:25.662-07:00',
   'organization_id': 'org1amxh23ednpz',
   'slot': None,
   'cover': None,
   'test_mode': False,
   'label': '423021.1',
   'location_id': 'loc1b68jjfjhatk',
   'shipment_id': 'sr1dqvp6p9z9h9p',
   'kit_request_id': None,
   'storage_condition': 'cold_80',
   'shipment_code': 'RTZ',
   'status': 'available',
   'expires_at': None,
   'aliquot_count': 93,
   'container_type': {'id': '96-pcr',
    'name': '96-Well twin.tec PCR Plate',
    'well_count': 96,
    'well_depth_mm': '14.6',
    'well_volume_ul': '160.0',
    'capabilities': ['magnetic_separate',
     'thermocycle',
     'qrt_thermocycle',
     'spin',
     'incubate',
     'seal',
     'image_plate',
     'sanger_sequence',
     'miniprep_destination',
     'stamp',
     'flash_freeze',
     'echo_dest',
     'magnetic_transfer_pcr',
     'bluewash',
     

In [5]:
[x['id'] for x in results['results']]

['ct1dqvp6nd7gej7', 'ct1dqvp6nm67z9b']

In [7]:
c = objects.Container(results['results'][0]['id'])

In [11]:
#c.aliquots.loc[c.aliquots['Name'] == 'UWBF_25784']
#c.aliquots.Name.unique()
col = 1
c.aliquots.loc[(c.aliquots.index + col) % 12  == 0]

,Name,Id,Volume,SBH_URI
Well Index,,,,
11,UWBF_24926,aq1dqvp6ndgjzkg,40:microliter,https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
23,UWBF_24952,aq1dqvp6ndrhdgp,40:microliter,https://hub.sd2e.org/user/sd2e/design/UWBF_24952/1
35,UWBF_24959,aq1dqvp6nef45yb,40:microliter,https://hub.sd2e.org/user/sd2e/design/UWBF_24959/1
47,UWBF_24960,aq1dqvp6nenwbrh,40:microliter,https://hub.sd2e.org/user/sd2e/design/UWBF_24960/1
59,UWBF_24961,aq1dqvp6newuqnq,40:microliter,https://hub.sd2e.org/user/sd2e/design/UWBF_24961/1
71,UWBF_24962,aq1dqvp6nf5nwew,40:microliter,https://hub.sd2e.org/user/sd2e/design/UWBF_24962/1
83,UWBF_25784,aq1dqvp6nfcg484,40:microliter,https://hub.sd2e.org/user/sd2e/design/CEN0x2EPK20x200x2D0x20MAT0x20A0x200x3A0x3A0x3A0x20pMOD4G0x2Ddcas90x2Dmxi10x20_X_0x20CEN0x2EPK20x200x2D0x20MAT0x20alpha0x200x3A0x3A0x3A0x20pMODKan0x2DHO0x2DpACT10x2DZEV40x200x7C0x20pMOD80x2DpGALZ40x2DeYFP/1


In [123]:
def get_matching_aliquots(strains, container_id, merge_key="Name"):
    c = objects.Container(container_id)
    strain_df = pd.DataFrame({merge_key : strains})
    c.aliquots['index'] = c.aliquots.index
    matches = c.aliquots.merge(strain_df, on=merge_key, how="inner")
    matches = matches.set_index('index')
    
    print("Matched:\n", str(matches.groupby(["Name"]).agg(len)))
    print("Didn't Match:\n", str([x for x in strains if x not in matches["Name"].unique()]))
    return matches
    
def get_src_wells_from_aliquots(container_id, matching_aliquots):
    print(matching_aliquots)
    return [ "{}/{}".format(container_id, x) for x in matching_aliquots.index.tolist() ]
    
def get_src_wells_matching_aliquots(strains, container_search_string=""):
    results = transcriptic.api.inventory(container_search_string)
    for result in results['results']:
        container_id = result['id']
        matching_aliquots = get_matching_aliquots(strains, container_id)
        if matching_aliquots is not None:
            src_wells = get_src_wells_from_aliquots(container_id, matching_aliquots)
            return src_wells
    return None
    


strains = [ 'UWBF_24926', 'UWBF_24952', 'UWBF_24959', 'UWBF_24960', 'UWBF_24961', 'UWBF_24962', 'UWBF_25784', 'UWBF_23970', 'UWBF_6390', 'UWBF_24864' ]
#strains = [  'UWBF_24952']

get_src_wells_matching_aliquots(strains, container_search_string=container_string)

Matched:
             Id  Volume  SBH_URI
Name                           
UWBF_23970  3   3       3      
UWBF_24864  3   3       3      
UWBF_24926  12  12      12     
UWBF_24952  12  12      12     
UWBF_24959  12  12      12     
UWBF_24960  12  12      12     
UWBF_24961  12  12      12     
UWBF_24962  12  12      12     
UWBF_25784  12  12      12     
UWBF_6390   3   3       3      
Didn't Match:
 []
             Name               Id         Volume                                             SBH_URI
index                                                                                                
0      UWBF_24926  aq1dqvp6ntrkkp5  40:microliter  https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
1      UWBF_24926  aq1dqvp6ntsqst6  40:microliter  https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
2      UWBF_24926  aq1dqvp6nttuyx7  40:microliter  https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
3      UWBF_24926  aq1dqvp6nttuyx8  40:microliter  https://hub.sd2e.org/user/s

['ct1dqvp6ntpb7e3/0',
 'ct1dqvp6ntpb7e3/1',
 'ct1dqvp6ntpb7e3/2',
 'ct1dqvp6ntpb7e3/3',
 'ct1dqvp6ntpb7e3/4',
 'ct1dqvp6ntpb7e3/5',
 'ct1dqvp6ntpb7e3/6',
 'ct1dqvp6ntpb7e3/7',
 'ct1dqvp6ntpb7e3/8',
 'ct1dqvp6ntpb7e3/9',
 'ct1dqvp6ntpb7e3/10',
 'ct1dqvp6ntpb7e3/11',
 'ct1dqvp6ntpb7e3/12',
 'ct1dqvp6ntpb7e3/13',
 'ct1dqvp6ntpb7e3/14',
 'ct1dqvp6ntpb7e3/15',
 'ct1dqvp6ntpb7e3/16',
 'ct1dqvp6ntpb7e3/17',
 'ct1dqvp6ntpb7e3/18',
 'ct1dqvp6ntpb7e3/19',
 'ct1dqvp6ntpb7e3/20',
 'ct1dqvp6ntpb7e3/21',
 'ct1dqvp6ntpb7e3/22',
 'ct1dqvp6ntpb7e3/23',
 'ct1dqvp6ntpb7e3/24',
 'ct1dqvp6ntpb7e3/25',
 'ct1dqvp6ntpb7e3/26',
 'ct1dqvp6ntpb7e3/27',
 'ct1dqvp6ntpb7e3/28',
 'ct1dqvp6ntpb7e3/29',
 'ct1dqvp6ntpb7e3/30',
 'ct1dqvp6ntpb7e3/31',
 'ct1dqvp6ntpb7e3/32',
 'ct1dqvp6ntpb7e3/33',
 'ct1dqvp6ntpb7e3/34',
 'ct1dqvp6ntpb7e3/35',
 'ct1dqvp6ntpb7e3/36',
 'ct1dqvp6ntpb7e3/37',
 'ct1dqvp6ntpb7e3/38',
 'ct1dqvp6ntpb7e3/39',
 'ct1dqvp6ntpb7e3/40',
 'ct1dqvp6ntpb7e3/41',
 'ct1dqvp6ntpb7e3/42',
 'ct1dqvp6ntpb7e3/43'

In [109]:
c = objects.Container("ct1dqvp6ntpb7e3")

In [115]:
c.

SyntaxError: invalid syntax (<ipython-input-115-cf463a5954f9>, line 1)

In [29]:
def get_design_df(design_file):
    j = json.load(open(design_file, 'r'))
    return pd.read_json(j['design'])
    
print(os.getcwd())
this_dir = os.getcwd()
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES_GROWTH_CURVE/experiments/growth_curve_test1/design_growth_curve_test1.json')))
df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES/experiments/dose_response_test1/design_dose_response_test1.json')))
df

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/pysd2cat/notebooks


,column_id,container,incubation_info.shaking,induction_info.dil_volume,induction_info.ind_media_volume,induction_info.induction_plate_info,induction_info.induction_time.doubling_time,induction_info.induction_time.total_induction_time,inoc_info.inc_time_1,inoc_info.inoc_media_vol,inoc_info.inoc_vol,inoc_info.inoculation_media,lab,measurement_type,plate_reader_info.fluor_em,plate_reader_info.fluor_ex,plate_reader_info.gain,protocol,recovery_info.recovery_media_vol,recovery_info.recovery_sample_vol,recovery_media,replicate,sampling_info.read_cult_vol,strain,temperature,batch,id,experiment_id,timepoint,output_id
0,0,ct1dqvp6nm67z9b,high_speed,1700:microliter,300:microliter,new-induction-plate,6:hour,30:hour,16:hour,700:microliter,10:microliter,sc_media,Strateos,PLATE_READER,509:nanometer,488:nanometer,0.274,timeseries,1000:microliter,10:microliter,sc_media,2,300:microliter,UWBF_24926,30,0,dose_response_test1_0_081dd002-0657-11ea-b3c1-0242ac110002,dose_response_test1,0,0
1,0,ct1dqvp6nm67z9b,high_speed,1700:microliter,300:microliter,new-induction-plate,6:hour,30:hour,16:hour,700:microliter,10:microliter,sc_media,Strateos,PLATE_READER,509:nanometer,488:nanometer,0.274,timeseries,1000:microliter,10:microliter,sc_media,2,300:microliter,UWBF_24926,30,0,dose_response_test1_0_081dd002-0657-11ea-b3c1-0242ac110002,dose_response_test1,6,0
2,0,ct1dqvp6nm67z9b,high_speed,1700:microliter,300:microliter,new-induction-plate,6:hour,30:hour,16:hour,700:microliter,10:microliter,sc_media,Strateos,PLATE_READER,509:nanometer,488:nanometer,0.274,timeseries,1000:microliter,10:microliter,sc_media,2,300:microliter,UWBF_24926,30,0,dose_response_test1_0_081dd002-0657-11ea-b3c1-0242ac110002,dose_response_test1,12,0
3,0,ct1dqvp6nm67z9b,high_speed,1700:microliter,300:microliter,new-induction-plate,6:hour,30:hour,16:hour,700:microliter,10:microliter,sc_media,Strateos,PLATE_READER,509:nanometer,488:nanometer,0.274,timeseries,1000:microliter,10:microliter,sc_media,2,300:microliter,UWBF_24926,30,0,dose_response_test1_0_081dd002-0657-11ea-b3c1-0242ac110002,dose_response_test1,18,0
4,0,ct1dqvp6nm67z9b,high_speed,1700:microliter,300:microliter,new-induction-plate,6:hour,30:hour,16:hour,700:microliter,10:microliter,sc_media,Strateos,PLATE_READER,509:nanometer,488:nanometer,0.274,timeseries,1000:microliter,10:microliter,sc_media,2,300:microliter,UWBF_24926,30,0,dose_response_test1_0_081dd002-0657-11ea-b3c1-0242ac110002,dose_response_test1,24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,11,ct1dqvp9jswdxfd,high_speed,1700:microliter,300:microliter,new-induction-plate,6:hour,30:hour,16:hour,700:microliter,10:microliter,sc_media,Strateos,PLATE_READER,509:nanometer,488:nanometer,0.274,timeseries,1000:microliter,10:microliter,None,4,300:microliter,UWBF_23970,35,4,dose_response_test1_4_0821ea52-0657-11ea-b3c1-0242ac110002,dose_response_test1,6,71
2372,11,ct1dqvp9jswdxfd,high_speed,1700:microliter,300:microliter,new-induction-plate,6:hour,30:hour,16:hour,700:microliter,10:microliter,sc_media,Strateos,PLATE_READER,509:nanometer,488:nanometer,0.274,timeseries,1000:microliter,10:microliter,None,4,300:microliter,UWBF_23970,35,4,dose_response_test1_4_0821ea52-0657-11ea-b3c1-0242ac110002,dose_response_test1,12,71
2373,11,ct1dqvp9jswdxfd,high_speed,1700:microliter,300:microliter,new-induction-plate,6:hour,30:hour,16:hour,700:microliter,10:microliter,sc_media,Strateos,PLATE_READER,509:nanometer,488:nanometer,0.274,timeseries,1000:microliter,10:microliter,None,4,300:microliter,UWBF_23970,35,4,dose_response_test1_4_0821ea52-0657-11ea-b3c1-0242ac110002,dose_response_test1,18,71
2374,11,ct1dqvp9jswdxfd,high_speed,1700:microliter,300:microliter,new-induction-plate,6:hour,30:hour,16:hour,700:microliter,10:microliter,sc_media,Strateos,PLATE_READER,509:nanometer,488:nanometer,0.274,timeseries,1000:microliter,10:microliter,None,4,300:microliter,UWBF_23970,35,4,dose_response_test1_4_0821ea52-0657-11ea-b3c1-0242

In [30]:
#f.loc[df.batch==1] #.drop(columns='id').drop_duplicates()
df.drop(columns='timepoint').drop_duplicates().groupby([  "strain"]).agg(len)
#df.measurement_type.unique()
#df.columns

,column_id,container,incubation_info.shaking,induction_info.dil_volume,induction_info.ind_media_volume,induction_info.induction_plate_info,induction_info.induction_time.doubling_time,induction_info.induction_time.total_induction_time,inoc_info.inc_time_1,inoc_info.inoc_media_vol,inoc_info.inoc_vol,inoc_info.inoculation_media,lab,measurement_type,plate_reader_info.fluor_em,plate_reader_info.fluor_ex,plate_reader_info.gain,protocol,recovery_info.recovery_media_vol,recovery_info.recovery_sample_vol,recovery_media,replicate,sampling_info.read_cult_vol,temperature,batch,id,experiment_id,output_id
strain,,,,,,,,,,,,,,,,,,,,,,,,,,,,
UWBF_22800,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12.0,12,12,12,12,12,12,12,12,12,12,12
UWBF_22801,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12.0,12,12,12,12,12,12,12,12,12,12,12
UWBF_23970,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15.0,15,15,15,15,15,15,15,15,15,15,15
UWBF_24864,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15.0,15,15,15,15,15,15,15,15,15,15,15
UWBF_24926,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24.0,24,24,24,24,24,24,24,24,24,24,24
UWBF_24952,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24.0,24,24,24,24,24,24,24,24,24,24,24
UWBF_24959,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48.0,48,48,48,48,48,48,48,48,48,48,48
UWBF_24960,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48.0,48,48,48,48,48,48,48,48,48,48,48
UWBF_24961,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48.0,48,48,48,48,48,48,48,48,48,48,48


In [26]:
from pysmt.shortcuts import Symbol, And, Not, is_sat, get_model

varA = Symbol("A") # Default type is Boolean
varB = Symbol("B")
f = And(varA, Not(varB))
model = get_model(f)
print(model)

A := True
B := False


In [8]:
from pysmt.shortcuts import Symbol, And, Not, is_sat, get_model, GE, LT, Int, String
from pysmt.typing import INT, StringType

samples = ["x1", "x2"]
tau_symbols = { x: Symbol("tau_{}".format(x), INT) for x in samples }

factors = { 
    "media" : {
        "domain" : ['m1', "m2"]
    }, 
    "strain" : {
        "domain" : [ "s1", "s2"]
    } 
}

sample_factors = { 
    sample: {
        factor : Symbol("{}({})".format(factor, sample), INT) for factor in factors 
    }  for sample in samples }

aliquots = ["a1", "a2"]
perp = Int(-1)
min_aliquot = Int(0)
max_aliquot = Int(len(aliquots))

tau_bounds = And([ And(GE(x, perp), LT(x, max_aliquot)) for k, x in tau_symbols.items() ])

factor_bounds = And([ And(GE(sample_factors[sample][factor], perp), LT(sample_factors[sample][factor], Int(len(factors[factor]['domain'])))) 
                  for sample in samples 
                  for factor in factors ])
#print(factor_bounds)



f = And( tau_bounds,
          factor_bounds)

model = get_model(f)
if model:
    print(model)
else:
    print("No solution found")

(((-1 <= 'media(x1)') & ('media(x1)' < 2)) & ((-1 <= 'strain(x1)') & ('strain(x1)' < 2)) & ((-1 <= 'media(x2)') & ('media(x2)' < 2)) & ((-1 <= 'strain(x2)') & ('strain(x2)' < 2)))
'strain(x2)' := -1
'media(x2)' := -1
'strain(x1)' := -1
'media(x1)' := -1
tau_x2 := -1
tau_x1 := -1
